# Exercise 14.9
## Hands-on Machine Learning with Scikit-Learn, Keras and Tensorflow, 2nd Ed

__Andrés Felipe García Albarracín <br>
Feb 8, 2021__ <br>

Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.

## 1. Libraries

In [1]:
import numpy as np
import time
import pandas as pd

In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 2. Load input data

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
mitad = int(x_test.shape[0]/2)
x_val = x_test[0:mitad,:,:]
x_test = x_test[mitad:,:,:]
y_val = y_test[0:mitad]
y_test = y_test[mitad:]

In [5]:
x_test.shape[1:]

(28, 28)

In [6]:
colResults = ["Model_name", "Description", "Loss", "Accuracy"]
dfResults = pd.DataFrame(columns=colResults)

## 3. Model proposals

### a) A model very similar to LeNet-5

In [7]:
inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 8, kernel_size = 5, strides = 2, padding = "same", activation = "relu")(inputs)
x = tf.keras.layers.AveragePooling2D(pool_size = 2, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 3, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(units = 10, activation = "softmax")(x)

In [8]:
model = tf.keras.Model(inputs = inputs, outputs = outputs)

In [9]:
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 8)         208       
_________________________________________________________________
average_pooling2d (AveragePo (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 16)          1168      
_________________________________________________________________
average_pooling2d_1 (Average (None, 3, 3, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          4160      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 1, 128)        

In [10]:
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'])

__Model training__

In [11]:
# =========================
# Training Parameters
# =========================
epochs = 100
patience = 10
# =========================
# Callbacks
# =========================
tb_cb = tf.keras.callbacks.TensorBoard("./Results_tensorboard/Ex_14_9/" + time.strftime("model_A_%Y_%m_%d_%H_%M_%S"))
es_cb = tf.keras.callbacks.EarlyStopping(patience= patience, restore_best_weights=True)
# =========================
# Training
# =========================
model.fit(
    x= x_train,
    y= y_train,
    epochs= epochs,
    callbacks= [tb_cb, es_cb],
    validation_data= (x_val, y_val))

Epoch 1/100
   1/1875 [..............................] - ETA: 0s - loss: 2.7541 - accuracy: 0.1250WARNING:tensorflow:From /home/data/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2433 - accuracy: 0.9216 - val_loss: 0.1154 - val_accuracy: 0.9654
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0912 - accuracy: 0.9723 - val_loss: 0.1080 - val_accuracy: 0.9678
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0762 - accuracy: 0.9778 - val_loss: 0.1080 - val_accuracy: 0.9732
Epoch 4/100
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0718 - accuracy: 0.9810 - val_loss: 0.0889 - val_accuracy: 0.9758
Epoch 5/100
1875/1875 [==========================

In [12]:
resultAct = model.evaluate(x = x_test, y = y_test)
print(resultAct)

157/157 [==============================] - 0s 1ms/step - loss: 0.0416 - accuracy: 0.9898
[0.04155489057302475, 0.989799976348877]


In [13]:
resultAct.insert(0,'A')
resultAct.insert(1,'Similar to LeNet')
dfResults = dfResults.append(
    pd.DataFrame(columns = colResults, data=[resultAct]),
    ignore_index=True)
dfResults

,Model_name,Description,Loss,Accuracy
0,A,Similar to LeNet,0.041555,0.9898


### b) Similar to LeNet and incluiding dropout 

In [14]:
inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 8, kernel_size = 5, strides = 2, padding = "same", activation = "relu")(inputs)
x = tf.keras.layers.AveragePooling2D(pool_size = 2, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 3, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Dropout(0.45)(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Dropout(0.45)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(units = 10, activation = "softmax")(x)

In [15]:
model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 8)         208       
_________________________________________________________________
average_pooling2d_2 (Average (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 16)          1168      
_________________________________________________________________
average_pooling2d_3 (Average (None, 3, 3, 16)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 3, 64)          4160      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 1, 1, 128)        

In [16]:
# =========================
# Compile model
# =========================
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'])
# =========================
# Training Parameters
# =========================
epochs = 100
patience = 10
# =========================
# Callbacks
# =========================
tb_cb = tf.keras.callbacks.TensorBoard("./Results_tensorboard/Ex_14_9/" + time.strftime("model_B_%Y_%m_%d_%H_%M_%S"))
es_cb = tf.keras.callbacks.EarlyStopping(patience= patience, restore_best_weights=True)
# =========================
# Training
# =========================
model.fit(
    x= x_train,
    y= y_train,
    epochs= epochs,
    callbacks= [tb_cb, es_cb],
    validation_data= (x_val, y_val))

Epoch 1/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5856 - accuracy: 0.8201 - val_loss: 0.2255 - val_accuracy: 0.9458
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2640 - accuracy: 0.9393 - val_loss: 0.1802 - val_accuracy: 0.9544
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2706 - accuracy: 0.9431 - val_loss: 0.2664 - val_accuracy: 0.9488
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2917 - accuracy: 0.9408 - val_loss: 0.2352 - val_accuracy: 0.9508
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3185 - accuracy: 0.9365 - val_loss: 0.2265 - val_accuracy: 0.9624
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3281 - accuracy: 0.9366 - val_loss: 0.3525 - val_accuracy: 0.9430
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3628 - accuracy: 0.9316 - val_loss: 0.2243 - val_ac

In [17]:
print(model.evaluate(x = x_train, y = y_train))
print(model.evaluate(x = x_val, y = y_val))

1875/1875 [==============================] - 3s 1ms/step - loss: 0.1366 - accuracy: 0.9668
[0.13664942979812622, 0.966783344745636]
157/157 [==============================] - 0s 1ms/step - loss: 0.1802 - accuracy: 0.9544
[0.18021848797798157, 0.9544000029563904]


In [18]:
resultAct = model.evaluate(x = x_test, y = y_test)
print(resultAct)

157/157 [==============================] - 0s 1ms/step - loss: 0.0896 - accuracy: 0.9782
[0.08964071422815323, 0.9782000184059143]


In [19]:
resultAct.insert(0,'B')
resultAct.insert(1,'Similar to LeNet including dropout at the final layers')
dfResults = dfResults.append(
    pd.DataFrame(columns = colResults, data=[resultAct]),
    ignore_index=True)
dfResults

,Model_name,Description,Loss,Accuracy
0,A,Similar to LeNet,0.041555,0.9898
1,B,Similar to LeNet including dropout at the fina...,0.089641,0.9782


### c) Local response normalization
Including some local response normalization layers at the beggining

In [20]:
inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 8, kernel_size = 5, strides = 2, padding = "same", activation = "relu")(inputs)
x = tf.keras.layers.AveragePooling2D(pool_size = 2, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 3, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(units = 10, activation = "softmax")(x)

In [21]:
model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 8)         208       
_________________________________________________________________
average_pooling2d_4 (Average (None, 7, 7, 8)           0         
_________________________________________________________________
lambda (Lambda)              (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 7, 7, 16)          1168      
_________________________________________________________________
average_pooling2d_5 (Average (None, 3, 3, 16)          0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 3, 3, 16)         

In [22]:
# =========================
# Compile model
# =========================
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'])
# =========================
# Training Parameters
# =========================
epochs = 100
patience = 10
# =========================
# Callbacks
# =========================
tb_cb = tf.keras.callbacks.TensorBoard("./Results_tensorboard/Ex_14_9/" + time.strftime("model_C_%Y_%m_%d_%H_%M_%S"))
es_cb = tf.keras.callbacks.EarlyStopping(patience= patience, restore_best_weights=True)
# =========================
# Training
# =========================
model.fit(
    x= x_train,
    y= y_train,
    epochs= epochs,
    callbacks= [tb_cb, es_cb],
    validation_data= (x_val, y_val))

Epoch 1/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2536 - accuracy: 0.9196 - val_loss: 0.1389 - val_accuracy: 0.9592
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0930 - accuracy: 0.9723 - val_loss: 0.1618 - val_accuracy: 0.9526
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0805 - accuracy: 0.9773 - val_loss: 0.0937 - val_accuracy: 0.9748
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0769 - accuracy: 0.9794 - val_loss: 0.1895 - val_accuracy: 0.9610
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0787 - accuracy: 0.9805 - val_loss: 0.1369 - val_accuracy: 0.9684
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0790 - accuracy: 0.9807 - val_loss: 0.1031 - val_accuracy: 0.9714
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0797 - accuracy: 0.9805 - val_loss: 0.1581 - val_ac

In [23]:
print(model.evaluate(x = x_train, y = y_train))
print(model.evaluate(x = x_val, y = y_val))

1875/1875 [==============================] - 3s 2ms/step - loss: 0.0566 - accuracy: 0.9838
[0.056593604385852814, 0.9837666749954224]
157/157 [==============================] - 0s 2ms/step - loss: 0.0937 - accuracy: 0.9748
[0.09369237720966339, 0.9747999906539917]


In [24]:
resultAct = model.evaluate(x = x_test, y = y_test)
print(resultAct)

157/157 [==============================] - 0s 2ms/step - loss: 0.0374 - accuracy: 0.9876
[0.037427883595228195, 0.9876000285148621]


In [25]:
resultAct.insert(0,'C')
resultAct.insert(1,'Including local responde normalization in the initial layers.')
dfResults = dfResults.append(
    pd.DataFrame(columns = colResults, data=[resultAct]),
    ignore_index=True)
dfResults

,Model_name,Description,Loss,Accuracy
0,A,Similar to LeNet,0.041555,0.9898
1,B,Similar to LeNet including dropout at the fina...,0.089641,0.9782
2,C,Including local responde normalization in the ...,0.037428,0.9876


### d) Going deeper
Including more convolutional layers with local response normalization

In [26]:
modelName = 'D'

inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 8, kernel_size = 5, strides = 2, padding = "same", activation = "relu")(inputs)
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = 5, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 2, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 3, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(units = 10, activation = "softmax")(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 14, 8)         208       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 16)        3216      
_________________________________________________________________
average_pooling2d_6 (Average (None, 7, 7, 16)          0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 7, 7, 32)         

In [27]:
# =========================
# Compile model
# =========================
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'])
# =========================
# Training Parameters
# =========================
epochs = 100
patience = 10
# =========================
# Callbacks
# =========================
tb_cb = tf.keras.callbacks.TensorBoard("./Results_tensorboard/Ex_14_9/" + time.strftime(f"model_{modelName}_%Y_%m_%d_%H_%M_%S"))
es_cb = tf.keras.callbacks.EarlyStopping(patience= patience, restore_best_weights=True)
# =========================
# Training
# =========================
model.fit(
    x= x_train,
    y= y_train,
    epochs= epochs,
    callbacks= [tb_cb, es_cb],
    validation_data= (x_val, y_val))

Epoch 1/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2153 - accuracy: 0.9331 - val_loss: 0.1244 - val_accuracy: 0.9608
Epoch 2/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0765 - accuracy: 0.9786 - val_loss: 0.0917 - val_accuracy: 0.9762
Epoch 3/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0679 - accuracy: 0.9821 - val_loss: 0.0789 - val_accuracy: 0.9776
Epoch 4/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0689 - accuracy: 0.9834 - val_loss: 0.0876 - val_accuracy: 0.9740
Epoch 5/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0676 - accuracy: 0.9837 - val_loss: 0.1095 - val_accuracy: 0.9764
Epoch 6/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0720 - accuracy: 0.9842 - val_loss: 0.0866 - val_accuracy: 0.9786
Epoch 7/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0728 - accuracy: 0.9841 - val_loss: 0.1225 - val_ac

In [28]:
print(model.evaluate(x = x_train, y = y_train))
print(model.evaluate(x = x_val, y = y_val))
resultAct = model.evaluate(x = x_test, y = y_test)
print(resultAct)

1875/1875 [==============================] - 3s 2ms/step - loss: 0.0532 - accuracy: 0.9850
[0.05319029465317726, 0.9850000143051147]
157/157 [==============================] - 0s 2ms/step - loss: 0.0789 - accuracy: 0.9776
[0.07887808233499527, 0.9775999784469604]
157/157 [==============================] - 0s 2ms/step - loss: 0.0319 - accuracy: 0.9914
[0.0319429486989975, 0.9914000034332275]


In [29]:
resultAct.insert(0,modelName)
resultAct.insert(1,'Including more layers in the LeNet architecture.')
dfResults = dfResults.append(
    pd.DataFrame(columns = colResults, data=[resultAct]),
    ignore_index=True)
dfResults

,Model_name,Description,Loss,Accuracy
0,A,Similar to LeNet,0.041555,0.9898
1,B,Similar to LeNet including dropout at the fina...,0.089641,0.9782
2,C,Including local responde normalization in the ...,0.037428,0.9876
3,D,Including more layers in the LeNet architecture.,0.031943,0.9914


### e) Including inception blocks

In [30]:
class inceptionLayer(tf.keras.layers.Layer):
    """
    Creates an inception layer following thar architecture of Google Net
    """
    def __init__(self, layer_filters = [8, 16, 4, 4, 12, 2], **kwargs):
        super().__init__(**kwargs)
        self.layers = [
            tf.keras.layers.Conv2D(filters = layer_filters[0], kernel_size = 1, strides = 1, padding = "same", activation = "relu"),
            tf.keras.layers.Conv2D(filters = layer_filters[1], kernel_size = 3, strides = 1, padding = "same", activation = "relu"),
            tf.keras.layers.Conv2D(filters = layer_filters[2], kernel_size = 5, strides = 1, padding = "same", activation = "relu"),
            tf.keras.layers.Conv2D(filters = layer_filters[3], kernel_size = 1, strides = 1, padding = "same", activation = "relu"),
            tf.keras.layers.Conv2D(filters = layer_filters[4], kernel_size = 1, strides = 1, padding = "same", activation = "relu"),
            tf.keras.layers.Conv2D(filters = layer_filters[5], kernel_size = 1, strides = 1, padding = "same", activation = "relu"),
            tf.keras.layers.MaxPooling2D(pool_size = 3, strides = 1, padding = "same")
        ]

    def call(self, inputs):
        path0 = self.layers[0](inputs)

        path1 = self.layers[4](inputs)
        path1 = self.layers[1](path1)

        path2 = self.layers[5](inputs)
        path2 = self.layers[2](path2)

        path3 = self.layers[6](inputs)
        path3 = self.layers[3](path3)

        final = tf.concat([path0, path1, path2, path3], axis = 3)

        return final




In [32]:
modelName = 'E'

inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 8, kernel_size = 5, strides = 2, padding = "same", activation = "relu")(inputs)
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = 5, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 2, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = inceptionLayer(layer_filters = [8, 16, 4, 4, 12, 2])(x)
x = inceptionLayer(layer_filters = [16, 32, 8, 8, 24, 4])(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 3, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(units = 10, activation = "softmax")(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 14, 14, 8)         208       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 14, 14, 16)        3216      
_________________________________________________________________
average_pooling2d_10 (Averag (None, 7, 7, 16)          0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 7, 7, 16)          0         
_________________________________________________________________
inception_layer_2 (inception (None, 7, 7, 32)          2390      
_________________________________________________________________
inception_layer_3 (inception (None, 7, 7, 64)        

In [33]:
# =========================
# Compile model
# =========================
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'])
# =========================
# Training Parameters
# =========================
epochs = 100
patience = 10
# =========================
# Callbacks
# =========================
tb_cb = tf.keras.callbacks.TensorBoard("./Results_tensorboard/Ex_14_9/" + time.strftime(f"model_{modelName}_%Y_%m_%d_%H_%M_%S"))
es_cb = tf.keras.callbacks.EarlyStopping(patience= patience, restore_best_weights=True)
# =========================
# Training
# =========================
model.fit(
    x= x_train,
    y= y_train,
    epochs= epochs,
    callbacks= [tb_cb, es_cb],
    validation_data= (x_val, y_val))

Epoch 1/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2519 - accuracy: 0.9201 - val_loss: 0.2122 - val_accuracy: 0.9356
Epoch 2/100
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0960 - accuracy: 0.9738 - val_loss: 0.0889 - val_accuracy: 0.9746
Epoch 3/100
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0884 - accuracy: 0.9772 - val_loss: 0.1104 - val_accuracy: 0.9744
Epoch 4/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0863 - accuracy: 0.9779 - val_loss: 0.0998 - val_accuracy: 0.9714
Epoch 5/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0882 - accuracy: 0.9782 - val_loss: 0.1428 - val_accuracy: 0.9660
Epoch 6/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1325 - accuracy: 0.9779 - val_loss: 0.0977 - val_accuracy: 0.9760
Epoch 7/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1007 - accuracy: 0.9780 - val_loss: 0.2983 -

In [34]:
print(model.evaluate(x = x_train, y = y_train))
print(model.evaluate(x = x_val, y = y_val))
resultAct = model.evaluate(x = x_test, y = y_test)
print(resultAct)

1875/1875 [==============================] - 4s 2ms/step - loss: 0.0620 - accuracy: 0.9822
[0.061971139162778854, 0.9821666479110718]
157/157 [==============================] - 0s 2ms/step - loss: 0.0889 - accuracy: 0.9746
[0.08886697888374329, 0.9746000170707703]
157/157 [==============================] - 0s 2ms/step - loss: 0.0382 - accuracy: 0.9894
[0.038159020245075226, 0.9894000291824341]


In [35]:
resultAct.insert(0,modelName)
resultAct.insert(1,'Including inception layers')
dfResults = dfResults.append(
    pd.DataFrame(columns = colResults, data=[resultAct]),
    ignore_index=True)
dfResults

,Model_name,Description,Loss,Accuracy
0,A,Similar to LeNet,0.041555,0.9898
1,B,Similar to LeNet including dropout at the fina...,0.089641,0.9782
2,C,Including local responde normalization in the ...,0.037428,0.9876
3,D,Including more layers in the LeNet architecture.,0.031943,0.9914
4,E,Including more layers in the LeNet architecture.,0.038159,0.9894


### f) Making the Inception architecture even deeper

In [40]:
modelName = 'F'

inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 8, kernel_size = 5, strides = 2, padding = "same", activation = "relu")(inputs)
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = 5, strides = 1, padding = "same", activation = "relu")(x)
x = tf.keras.layers.AveragePooling2D(pool_size = 2, strides = 2, padding = "valid")(x)
x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = inceptionLayer(layer_filters = [8, 16, 4, 4, 12, 2])(x)
x = inceptionLayer(layer_filters = [16, 32, 8, 8, 24, 4])(x)
x = tf.keras.layers.MaxPooling2D(pool_size = 3, strides = 2, padding = "valid")(x)
#x = tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1, alpha=0.00002, beta=0.75))(x)
x = inceptionLayer(layer_filters = [32, 64, 16, 16, 48, 8])(x)
x = inceptionLayer(layer_filters = [64, 128, 32, 32, 96, 16])(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 1, strides = 1, padding = "valid", activation = "relu")(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(units = 10, activation = "softmax")(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 14, 14, 8)         208       
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 14, 14, 16)        3216      
_________________________________________________________________
average_pooling2d_14 (Averag (None, 7, 7, 16)          0         
_________________________________________________________________
lambda_10 (Lambda)           (None, 7, 7, 16)          0         
_________________________________________________________________
inception_layer_12 (inceptio (None, 7, 7, 32)          2390      
_________________________________________________________________
inception_layer_13 (inceptio (None, 7, 7, 64)        

In [41]:
# =========================
# Compile model
# =========================
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'])
# =========================
# Training Parameters
# =========================
epochs = 100
patience = 10
# =========================
# Callbacks
# =========================
tb_cb = tf.keras.callbacks.TensorBoard("./Results_tensorboard/Ex_14_9/" + time.strftime(f"model_{modelName}_%Y_%m_%d_%H_%M_%S"))
es_cb = tf.keras.callbacks.EarlyStopping(patience= patience, restore_best_weights=True)
# =========================
# Training
# =========================
model.fit(
    x= x_train,
    y= y_train,
    epochs= epochs,
    callbacks= [tb_cb, es_cb],
    validation_data= (x_val, y_val))

Epoch 1/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2821 - accuracy: 0.9123 - val_loss: 0.1215 - val_accuracy: 0.9640
Epoch 2/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0959 - accuracy: 0.9721 - val_loss: 0.0962 - val_accuracy: 0.9716
Epoch 3/100
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0825 - accuracy: 0.9780 - val_loss: 0.1297 - val_accuracy: 0.9642
Epoch 4/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0812 - accuracy: 0.9791 - val_loss: 0.1187 - val_accuracy: 0.9678
Epoch 5/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0888 - accuracy: 0.9804 - val_loss: 0.1285 - val_accuracy: 0.9752
Epoch 6/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0788 - accuracy: 0.9809 - val_loss: 0.1073 - val_accuracy: 0.9704
Epoch 7/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0750 - accuracy: 0.9812 - val_loss: 0.0940 -

In [42]:
print(model.evaluate(x = x_train, y = y_train))
print(model.evaluate(x = x_val, y = y_val))
resultAct = model.evaluate(x = x_test, y = y_test)
print(resultAct)

1875/1875 [==============================] - 5s 3ms/step - loss: 0.0344 - accuracy: 0.9902
[0.03438741713762283, 0.9902333617210388]
157/157 [==============================] - 0s 3ms/step - loss: 0.0608 - accuracy: 0.9812
[0.06078333035111427, 0.9811999797821045]
157/157 [==============================] - 1s 3ms/step - loss: 0.0241 - accuracy: 0.9932
[0.02412254922091961, 0.9932000041007996]


In [43]:
resultAct.insert(0,modelName)
resultAct.insert(1,'Making the inception layers even deeper')
dfResults = dfResults.append(
    pd.DataFrame(columns = colResults, data=[resultAct]),
    ignore_index=True)
dfResults

,Model_name,Description,Loss,Accuracy
0,A,Similar to LeNet,0.041555,0.9898
1,B,Similar to LeNet including dropout at the fina...,0.089641,0.9782
2,C,Including local responde normalization in the ...,0.037428,0.9876
3,D,Including more layers in the LeNet architecture.,0.031943,0.9914
4,E,Including inception layers,0.038159,0.9894
5,F,Making the inception layers even deeper,0.024123,0.9932
